## Begin creating algorithm to solve 2025 paper example by decomposition

In [ ]:
# imports

import hypernetx as hnx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import warnings 
#init_printing(use_unicode=True)
warnings.simplefilter('ignore')

from sympy import *

init_printing()




In [55]:
# Everything we need to define our reaction network:

SM_1 = Matrix( [[-1, 1, 0], [1, -1, -1], [0, 0, 1], [-1, 0, -1], [0, 1, 0]] ) # The SM of module 1

num_internal_species = 3 # for extracting the internal and external SMs

In [ ]:


# Define our class:

class stoich_matrix:

    #==========================================================================================================================================
    # INIT
    #

    def __init__(self, SM, num_internal_species):

        #
        # Define some objects to hold the full SM, internal and external SMs:
        #

        self.matrix = SM # Returns the passed SM, ready for print

        self.internal_SM = SM[0:num_internal_species, :] # finds the internal species SM by selecting the number of rows needed

        self.external_SM = SM[num_internal_species: len(SM), :] # finds SM for external species using remaning rows after internal species

        self.module_1_labels = {0: 'Ea', 1: 'EaS', 2: 'EaS2', 3: 'S', 4: 'Na'} # for the hypergraphs

    
    #==========================================================================================================================================
    # HYPERGRAPHS
    #
    # Define a new function to find the hypergraphs for the internal species
    #

    def hypergraph_internals(self):


        internals_HG = hnx.Hypergraph.from_incidence_matrix(self.internal_SM) # create hypergraph, using the internal SM defined in self

        hnx.draw(internals_HG, node_labels=self.module_1_labels, with_edge_labels=True) # print this using the labels defined in self

    #
    # Define a new function to find the hypergraphs for the full stoichiometric matrix
    #

    def hypergraph_full(self):

        full_HG = hnx.Hypergraph.from_incidence_matrix(self.matrix) # create hypergraph, using the full SM defined in self

        hnx.draw(full_HG, node_labels=self.module_1_labels, with_edge_labels=True) # print this using the labels defined in self

    #==========================================================================================================================================
    # REACTION LEVEL CYCLES
    #

    def reaction_cycles_matrix(self):
        
        reaction_cycles = (self.internal_SM).nullspace() # finds the kernel for the SM internal

        # Check if there are any cycles:

        if not reaction_cycles:

            print("No internal cycles. Kernel is empty.")

        # build cycle matrix from kernel vectors if kernel is NOT empty

        else:

            cycle_matrix = reaction_cycles[0] # add first vector to cycle matrix so we can add rest later

            for cycle in reaction_cycles[1:]: # starting at second vector in kernel

                cycle_matrix = cycle_matrix.row_join(cycle) # connect vectors from kernel column-wise, row_join puts elemetns of adjacent vectors together


            return cycle_matrix
        
    #==========================================================================================================================================
    # CONSERVATION LAW MATRIX
    #
    def conservation_laws_matrix(self):

        cokernel_SM = (self.matrix.T).nullspace() # finds the cokernel of the full SM

        if not cokernel_SM:

            print("Cokernel of Stoichiometric Matrix empty. No conservation laws.")

        else:

            cons_laws = cokernel_SM[0] # adds first element of cokernel

            for vec in cokernel_SM[1:]: # add vectors from next row onwards

                cons_laws = cons_laws.row_join(vec)

            return cons_laws.T # to put in same form as paper, transpose matrix


        






        


        

mod1 = stoich_matrix(SM_1, num_internal_species)


⎡1   1   1  0  0⎤
⎢               ⎥
⎣-2  -1  0  1  1⎦

In [127]:
stoich_matrix_2 = Matrix([[-1, 0, 1, 0, 0], # first row of internal
                           [1, -1, 0, -1, 0], 
                           [0, 1, -1, 0, 1], 
                           [0, 0, 0, 1, -1],
                             [0, 0, 0, -1, 0], # first row external 
                             [0, 0, 0, 0, 1],
                             [-1, 0, 0, 0, 0],
                             [0, 0, 1, 0, 0]])


mod2 = stoich_matrix(stoich_matrix_2, 4)

mod2.conservation_laws_matrix()

⎡1   1   1   1  0  0  0  0⎤
⎢                         ⎥
⎢-1  -1  -1  0  1  1  0  0⎥
⎢                         ⎥
⎣-1  0   0   0  0  0  1  1⎦